# Identify Datasets

This notebook is a part of the [Crop Segmentation and Classification](crop-segmentation-and-classification.ipynb) notebook project. In this notebook, we identify the ground truth data and imagery data for use in crop segmentation and classification.

The general sections are:
1. [Explore Ground Truth Data](#ground_truth)
2. [Identify Area of Interest](#aoi)
3. [Download Planet Scene](#pl_scene)

Ground truth crop type and boundary data is not easy to come by. Therefore, the area and time of interest for this problem is primarily defined by the availability of ground truth data. The [2015 Sacramento County Land Use DWR Survey Dataset](http://www.water.ca.gov/landwateruse/lusrvymain.cfm) is a free dataset covering Sacramento county in 2015. It provides hand-adjusted boundaries and provides crop types.

The satellite imagery we will use in this study are Landsat 8 and SSO Planetscope 2 imagery. SSO Planetscope 2 satellites were launched Feb 14, 2017 ([news release](https://www.planet.com/pulse/planet-launches-satellite-constellation-to-image-the-whole-planet-daily/)), therefore they did not image Sacramento county in 2015. Landsat 8 will be used as a bridge between 2015 and 2017.

### Usage Notes

This notebook was developed in a Docker container. This [Dockerfile](Dockerfile) was used to build the image.

The user-specific planet api key must be stored in the environmental variable $PL_API_KEY. To pass this key into the Docker container, add -e PL_API_KEY=your_api_key when you call docker run (replace 'your_api_key' with your api key).

In [1]:
# Notebook dependencies
from __future__ import print_function

import copy
import datetime
from functools import partial
import json
import os

import ipyleaflet as ipyl
import ipywidgets as ipyw
from IPython.display import display, Image
import fiona
import pandas as pd
from planet import api
from planet.api import filters
import pyproj
import shapely
from shapely.geometry import shape, mapping
from shapely.ops import transform

In [2]:
!conda env export -n python2

name: python2
channels:
- conda-forge
- defaults
dependencies:
- affine=2.1.0=py27_0
- asn1crypto=0.22.0=py27_0
- backports.shutil_get_terminal_size=1.0.0=py27_1
- backports_abc=0.5=py27_0
- beautifulsoup4=4.5.3=py27_0
- bkcharts=0.2=py27_0
- blas=1.1=openblas
- bleach=1.5.0=py27_0
- bokeh=0.12.6=py27_0
- boost=1.64.0=np112py27_2
- boost-cpp=1.64.0=1
- boto3=1.4.4=py27_0
- botocore=1.5.84=py27_0
- bzip2=1.0.6=1
- ca-certificates=2017.4.17=0
- cairo=1.14.6=4
- certifi=2017.4.17=py27_0
- cffi=1.10.0=py27_0
- chardet=3.0.2=py27_1
- click=6.7=py27_0
- click-plugins=1.0.3=py27_0
- cligj=0.4.0=py27_0
- cloudpickle=0.2.2=py27_4
- configparser=3.5.0=py27_0
- cryptography=1.9=py27_0
- curl=7.52.1=0
- cycler=0.10.0=py27_0
- cython=0.25.2=py27_1
- dask=0.15.0=py27_0
- dbus=1.10.10=3
- decorator=4.0.11=py27_0
- dill=0.2.6=py27_0
- distributed=1.17.1=py27_0
- docutils=0.13.1=py27_0
- entrypoints=0.2.3=py27_1
- enum34=1.1.6=py27_1
- expat=2.1.0=2
- fastcache=1.0.2=py27_0
- ffmpeg=3.2.4=1
- fiona=1.7

<a id='ground_truth'></a>

## Explore Ground Truth Data

#### Downloading data
To obtain and prepare the ground truth data, download the shapefile zip file from (http://www.water.ca.gov/landwateruse/docs/landusedata/shapes/15sa.zip) and then unzip into /data.

Execute the following commands in `crop-classification` directory:
```
mkdir data
mkdir data/dwr_survey
wget http://www.water.ca.gov/landwateruse/docs/landusedata/shapes/15sa.zip
unzip 15sa.zip -d data/dwr_survey/
```

expected output:
```
Archive:  15sa.zip
  inflating: data/dwr_survey/SA15.dbf  
  inflating: data/dwr_survey/SA15.jgw  
  inflating: data/dwr_survey/SA15.jpg  
  inflating: data/dwr_survey/SA15.prj  
  inflating: data/dwr_survey/SA15.shp  
  inflating: data/dwr_survey/SA15.shp.xml  
  inflating: data/dwr_survey/SA15.shx  
  inflating: data/dwr_survey/DWR Data Disclaimer.pdf  
  inflating: data/dwr_survey/SA15Meta.pdf  
  inflating: data/dwr_survey/attributes_rev02102010.pdf  
  inflating: data/dwr_survey/09legend.pdf  
```

In [2]:
# Let's specify the shapefile location and ensure it indeed exists
survey_shapefile = 'data/dwr_survey/SA15.shp'
assert os.path.isfile(survey_shapefile)

#### Load the survey shapefile

We will use [fiona](http://toblerity.org/fiona/manual.html) to load the shapefile, [shapely](http://toblerity.org/shapely/manual.html) to manage the geometries

##### Reproject to WGS84

What is the coordinate reference system for this dataset?

In [3]:
with fiona.open(survey_shapefile, 'r') as survey_data:
    src_proj = survey_data.crs['init']
print(src_proj)

epsg:26910


Turns out it is [EPSG:26910](http://spatialreference.org/ref/epsg/26910/). Geojson only supports [EPSG:4326](http://spatialreference.org/ref/epsg/4326/). We will need to reproject the shapes.

In [4]:
# define projection
# from shapely [docs](http://toblerity.org/shapely/manual.html#shapely.ops.transform)
dst_proj = 'epsg:4326'
project_to_wkt = partial(
    pyproj.transform,
    pyproj.Proj(init=src_proj),
    pyproj.Proj(init=dst_proj))

def project_feature(feat):
    g1 = shape(feat['geometry'])
    g2 = transform(project_to_wkt, g1)
    feat['geometry'] = mapping(g2)

##### Filter survey data to agricultural classes

The survey data has attributes that provide the crop type. These attributes are described in a pdf distributed with the shapefile. It was unzipped along with the shapefile files and is located at `data/dwr_survey/09legend.pdf`.

We are interested in the agricultural classes. Class is specified by the 'CLASS1' attribute of the feature and agricultural classes are: G, R, F, P, T, D, C, and V.

In [5]:
# Class ids from dwr_survey/09legend.pdf
agg_classes = ['G', 'R', 'F', 'P', 'T', 'D', 'C', 'V']

def is_agricultural(feat):
    return feat['properties']['CLASS1'] in agg_classes

#### Finally: Load data

In [6]:
features = []
with fiona.open(survey_shapefile) as survey_data:
    for feat in survey_data:
        if is_agricultural(feat):
            project_feature(feat)
            features.append(feat)
print(len(features))

7429


#### Save Filtered Ground Truth Data

Save as geojson file

In [7]:
ground_truth_file = os.path.join('data', 'ground-truth.geojson')
with open(ground_truth_file, "w") as f:
    f.write(json.dumps(features))
print(ground_truth_file)

data/ground-truth.geojson


#### Explore data on map

In [8]:
# Assign colors to classes
# colors determined using [colorbrewer2.org](http://colorbrewer2.org/#type=sequential&scheme=BuGn&n=3)
colors = ['#ffffd9','#edf8b1','#c7e9b4','#7fcdbb','#41b6c4','#1d91c0','#225ea8','#0c2c84']
class_colors = dict((a,c) for a,c in zip(agg_classes, colors))
def get_color(cls):
    return class_colors[cls]

In [9]:
# Initialize map, deleting map instance if it exists
try:
    del filtered_map
except NameError:
    pass

zoom = 9
center = [38.3586252, -121.3853994] # lat/lon
bounds = None #[(38.23467444211417, -121.66706085205077), (38.342464018725714, -121.32030487060547)]

In [10]:
# Reuse map parameters if map already defined
try:
    center = filtered_map.center
    print(center)
    zoom = filtered_map.zoom
    print(zoom)
    bounds = filtered_map.bounds
    print(bounds)
except NameError:
    pass

# Create the map
# Change tile layer to one that makes it easier to see crop features
# Layer selected using https://leaflet-extras.github.io/leaflet-providers/preview/
map_tiles = ipyl.TileLayer(url='http://{s}.basemaps.cartocdn.com/light_all/{z}/{x}/{y}.png')
filtered_map = ipyl.Map(
        center=center, 
        zoom=zoom,
        default_tiles = map_tiles
    )

if bounds is not None:
    # create bound box geometry from map bounds readout
    bounds_box = shapely.geometry.box(bounds[0][1], bounds[0][0],
                                      bounds[1][1], bounds[1][0])

    # display bounds box
    bounds_feature = {
        'geometry':mapping(bounds_box),
        "properties":{
                'style':{'color': 'red', 'fillOpacity': 0.0, 'weight': 1}},
        'type':u"Feature"}

    bounds_layer = ipyl.GeoJSON(data=bounds_feature)
    filtered_map.add_layer(bounds_layer)

    # filter to bounds
    display_features = [f for f in features if shape(f['geometry']).within(bounds_box)]
    
else:
    display_features = features

feature_collection = {
    "type": "FeatureCollection",
    "features": display_features
}

for f in feature_collection['features']:
    feature_color = get_color(f['properties']['CLASS1'])
    f['properties']['style'] = {
        'weight': 0,
        'fillColor': feature_color,
        'fillOpacity': 1}

feature_layer = ipyl.GeoJSON(data=feature_collection)

# Display class as label
label = ipyw.Label(layout=ipyw.Layout(width='100%'))
def hover_handler(event=None, id=None, properties=None):
    label.value = 'class: {}'.format(properties['CLASS1'])

feature_layer.on_hover(hover_handler)
filtered_map.add_layer(feature_layer)   

# Display
ipyw.VBox([filtered_map, label])

# NOTE: it may take a little while for this to display

Widget Javascript not detected.  It may not be installed or enabled properly.


<a id='aoi'></a>

## Identify Area of Interest

Selection of the area of interest for our study is based on the following:
1. compact representation of many crop classes
2. availability of imagery
3. as large as possible but smaller than a planet image (to allow for in-scene analysis)

### Criteria 1: compact representation of many crop classes

Let's start by identifying a region of compact representation of many crop classes. We will do so by drawing a box over the map of the crops and then displaying the number of unique classes represented in the box.

In [11]:
# Initialize map using parameters from above map
# and deleting map instance if it exists
try:
    del aoi_map
except NameError:
    pass

center = filtered_map.center
zoom = filtered_map.zoom
bounds = filtered_map.bounds
default_tiles = filtered_map.default_tiles

In [12]:
# Create map, adding box drawing controls
# Reuse parameters if map already exists
try:
    center = aoi_map.center
    zoom = aoi_map.zoom
    bounds = aoi_map.bounds
except NameError:
    pass

# Create the map
aoi_map = ipyl.Map(
        center=center, 
        zoom=zoom,
        default_tiles = map_tiles
    )

# Add box drawing control
# refs:
# https://github.com/kscottz/PythonFromSpace/blob/master/TheBasics.ipynb
# https://github.com/ellisonbg/ipyleaflet/blob/master/examples/DrawControl.ipynb
rectangle = {'shapeOptions': {'color': 'blue'}} 
dc = ipyl.DrawControl(
    polygon={}, polyline={}, # disable polygons and polylines
    rectangle={'shapeOptions': {'color': 'blue'}}
)

# When a box is drawn, update the label with the number of unique classes
# and save the box geometry as AOI
total_unique_classes = len(set([f['properties']['CLASS1'] for f in features]))
label = ipyw.Label(layout=ipyw.Layout(width='100%'))
aoi = None
def handle_draw(self, action, geo_json):
    box_shape = shape(geo_json['geometry'])
    contained_features = [f for f in features
                          if shape(f['geometry']).within(box_shape)]
    unique_classes = set([f['properties']['CLASS1'] for f in contained_features])
    label.value = '{} unique classes out of {} total'.format(
        len(unique_classes), total_unique_classes)
    global aoi
    aoi = geo_json
    
dc.on_draw(handle_draw)
aoi_map.add_control(dc)

# Add crop type features
feature_collection = {
    "type": "FeatureCollection",
    "features": features
}

for f in feature_collection['features']:
    feature_color = get_color(f['properties']['CLASS1'])
    f['properties']['style'] = {
        'weight': 0,
        'fillColor': feature_color,
        'fillOpacity': 1}

feature_layer = ipyl.GeoJSON(data=feature_collection)
aoi_map.add_layer(feature_layer)   

# Display map and label
ipyw.VBox([aoi_map, label])

Widget Javascript not detected.  It may not be installed or enabled properly.


In [13]:
# Run this to use cached aoi
aoi = {u'geometry': {u'type': u'Polygon', u'coordinates': [[[-121.58460974693298, 38.29170496647727], [-121.58460974693298, 38.32726528409606], [-121.5248715877533, 38.32726528409606], [-121.5248715877533, 38.29170496647727], [-121.58460974693298, 38.29170496647727]]]}, u'type': u'Feature', u'properties': {u'style': {u'opacity': 0.5, u'noClip': False, u'weight': 4, u'fillColor': None, u'color': u'blue', u'lineCap': None, u'stroke': True, u'smoothFactor': 1, u'dashArray': None, u'fillOpacity': 0.2, u'clickable': True, u'lineJoin': None, u'fill': True}}}

### Criteria 2: availability of imagery

How many PL images cover the AOI defined above?

We will answer this question by querying the planet API. This requires a user-specific planet api key. The planet api key is pulled from the environment variable PLANET_API_KEY.

The client is how we interact with the planet api. It is created with the user-specific api key, which is pulled from $PL_API_KEY

[planet client documentation](https://planetlabs.github.io/planet-client-python/index.html) 

Much of this code is pulled from [PythonFromSpace/TheBasics.ipynb](https://github.com/kscottz/PythonFromSpace/blob/master/TheBasics.ipynb)

In [14]:
planet_api_key = os.environ['PL_API_KEY']

# quick check that key is defined
assert planet_api_key, "PL_API_KEY not defined."

client = api.ClientV1(api_key=planet_api_key)

Query Planet API

In [15]:
# build a query using the AOI and
# a cloud_cover filter that excludes 'cloud free' scenes
old = datetime.datetime(year=2015,month=1,day=1)

search_aoi = aoi['geometry']
query = filters.and_filter(
    filters.geom_filter(search_aoi),
    filters.range_filter('cloud_cover', lt=50),
    filters.date_range('acquired', gt=old)
)

# build a request for only PlanetScope imagery
request = filters.build_search_request(
    query, item_types=['PSScene4Band']
)

# run search
# if you don't have an API key configured, this will raise an exception
result = client.quick_search(request)
scenes = []
planet_map = {}
for item in result.items_iter(limit=500):
    planet_map[item['id']]=item
    props = item['properties']
    props["id"] = item['id']
    props["geometry"] = item["geometry"]
    props["thumbnail"] = item["_links"]["thumbnail"]
    scenes.append(props)
scenes = pd.DataFrame(data=scenes)
print(len(scenes))

211


Cleanup and calculate overlap

In [16]:
# now let's clean up the datetime stuff
# make a shapely shape from our aoi
aoi_shape = shape(search_aoi)
footprints = []
overlaps = []
# go through the geometry from our api call, convert to a shape and calculate overlap area.
# also save the shape for safe keeping
for footprint in scenes["geometry"].tolist():
    s = shape(footprint)
    footprints.append(s)
    overlap = 100.0*(aoi_shape.intersection(s).area / aoi_shape.area)
    overlaps.append(overlap)
# take our lists and add them back to our dataframe
scenes['overlap'] = pd.Series(overlaps, index=scenes.index)
scenes['footprint'] = pd.Series(footprints, index=scenes.index)
# now make sure pandas knows about our date/time columns.
scenes["acquired"] = pd.to_datetime(scenes["acquired"])
scenes["published"] = pd.to_datetime(scenes["published"])
scenes["updated"] = pd.to_datetime(scenes["updated"])
# scenes.head()

Filter to scenes that contain AOI

If the number is zero, go back and redefine the AOI to be smaller.

In [17]:
full_coverage = scenes["overlap"] > 99
good_scenes = scenes[(full_coverage)]
print(len(good_scenes))

34


In [18]:
# write aoi to geojson file
aoi_file = os.path.join('data', 'aoi.geojson')
with open(aoi_file,"w") as f:
    f.write(json.dumps(aoi))
print(aoi_file)

data/aoi.geojson


Visualize footprints

In [19]:
center = aoi_map.center
zoom = aoi_map.zoom - 1
bounds = aoi_map.bounds
default_tiles = aoi_map.default_tiles

# Create the map
footprints_map = ipyl.Map(
        center=center, 
        zoom=zoom,
        default_tiles = default_tiles
    )


# Add footprint features as layer
footprints = good_scenes["geometry"].tolist()
footprint_collection = {
    "type": "FeatureCollection",
    "features": footprints
}

footprint_layer = ipyl.GeoJSON(data=footprint_collection)
footprints_map.add_layer(footprint_layer)  

# Add aoi as layer, make it a different color and with opaque fill
aoi_feature = copy.deepcopy(aoi)
aoi_feature['properties']['style'] = {
    'color': 'red',
    'fillOpacity':0.1
}
aoi_layer = ipyl.GeoJSON(data=aoi_feature)
footprints_map.add_layer(aoi_layer)   

# Display map and label
ipyw.VBox([footprints_map])

Widget Javascript not detected.  It may not be installed or enabled properly.


Visualize subest of scene thumbnails

If all you see are broken image icons, click one of the urls and log into the site.

In [20]:
imgs = []
# loop through our thumbnails and add display them
for thumb_url in good_scenes['thumbnail'].tolist()[:3]:
    img_image = Image(url=thumb_url)
    display(img_image)
    print(thumb_url)

https://api.planet.com/data/v1/item-types/PSScene4Band/items/20170709_180702_100e/thumb


https://api.planet.com/data/v1/item-types/PSScene4Band/items/20170702_180740_0f10/thumb


https://api.planet.com/data/v1/item-types/PSScene4Band/items/20170701_180629_101e/thumb


<a id='pl_scene'></a>

## Download Planet Scene

The scene with id [20160831_180231_0e0e](https://api.planet.com/data/v1/item-types/PSScene4Band/items/20160831_180231_0e0e/thumb) looks pretty good. It is from August 2016 and looks pretty clear. 

We will download this scene and its metadata using the planet client cli. This approach requires much less code than using the python bindings.

In [21]:
analysis_thumbnail = 'https://api.planet.com/data/v1/item-types/PSScene4Band/items/20160831_180231_0e0e/thumb'

In [22]:
img_image = Image(url=analysis_thumbnail)
display(img_image)

In [23]:
!planet data download --item-type PSScene4Band --dest data \
    --asset-type analytic,analytic_xml --string-in id 20160831_180231_0e0e

activating: 0            complete: 0              elapsed: 0                    
paging: False            pending: 0                                             

activating: 1            complete: 0              downloaded: 0.00MB            
downloading: 0           elapsed: 1               paging: True                  
pending: 0                                                                      

activating: 1            complete: 0              downloaded: 0.00MB            
downloading: 0           elapsed: 2               paging: False                 
pending: 0                                                                      

activating: 0            complete: 0              downloaded: 0.00MB            
downloading: 1           elapsed: 3               paging: False                 
pending: 1                                                                      

activating: 0            complete: 0              downloaded: 0.00MB            
downloading: 2          

The image and metadata are now saved and are ready to be processed in other notebooks.